In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

import os

**통계청 행정구역 정보 import**

In [3]:
district = pd.read_excel('.//통계청_행정구역//adm_code.xls', sheet_name='2015년', skiprows=1)
district

,시도코드,시도명칭,시군구코드,시군구명칭,읍면동코드,읍면동명칭
0,11,서울특별시,11010,종로구,1101053,사직동
1,11,서울특별시,11010,종로구,1101054,삼청동
2,11,서울특별시,11010,종로구,1101055,부암동
3,11,서울특별시,11010,종로구,1101056,평창동
4,11,서울특별시,11010,종로구,1101057,무악동
...,...,...,...,...,...,...
3497,39,제주특별자치도,39020,서귀포시,3902058,서홍동
3498,39,제주특별자치도,39020,서귀포시,3902059,대륜동
3499,39,제주특별자치도,39020,서귀포시,3902060,대천동
3500,39,제주특별자치도,39020,서귀포시,3902061,중문동


In [4]:
district = district.rename(columns={'시도명칭':'sido_name','시군구명칭':'sigungu_name','읍면동명칭':'dong_name',
                                    '시도코드':'sido_CD','시군구코드':'sigungu_CD','읍면동코드':'dong_CD'})

district['sidosigungu_name'] = district['sido_name'] +' '+ district['sigungu_name']
district['sidosigungudong_name'] = district['sido_name'] +' '+ district['sigungu_name'] +' '+ district['dong_name']

district.head()

,sido_CD,sido_name,sigungu_CD,sigungu_name,dong_CD,dong_name,sidosigungu_name,sidosigungudong_name
0,11,서울특별시,11010,종로구,1101053,사직동,서울특별시 종로구,서울특별시 종로구 사직동
1,11,서울특별시,11010,종로구,1101054,삼청동,서울특별시 종로구,서울특별시 종로구 삼청동
2,11,서울특별시,11010,종로구,1101055,부암동,서울특별시 종로구,서울특별시 종로구 부암동
3,11,서울특별시,11010,종로구,1101056,평창동,서울특별시 종로구,서울특별시 종로구 평창동
4,11,서울특별시,11010,종로구,1101057,무악동,서울특별시 종로구,서울특별시 종로구 무악동


**폴더 내 파일 import**

2015년 merge

In [5]:
import copy

# 2015년 sido dataset
files =  os.listdir('.//2015년//2015_sido')
files

for idx  in ['EVI','NDVI']:
    
    d = pd.DataFrame({'name':['Chungcheongbuk-do', 'Chungcheongnam-do', 'Gangwon-do',
       'Gyeonggi-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju',
       'Jeollabuk-do', 'Jeollanam-do', 'Busan', 'Daegu', 'Daejeon',
       'Gwangju', 'Incheon', 'Ulsan', 'Seoul', 'Sejong']})
    
    means =  [c for c in files if idx in c and 'mean' in c]
    
    for m in means:
        v = [c for c in files if idx in c and c.startswith(m.split('_')[0])][0]
    
        a = pd.read_csv('.//2015년//2015_sido' + '//' + m, skiprows=1).rename(columns={'Unnamed: 0':'name', 'Unnamed: 1':f"{idx}_{m.split('_')[0]}_SIDO_avg"})
        b = pd.read_csv('.//2015년//2015_sido' + '//' + v, skiprows=1).rename(columns={'Unnamed: 0':'name', 'Unnamed: 1':f"{idx}_{v.split('_')[0]}_SIDO_std"})

        c = pd.merge(left=a, right=b, on='name', how='outer')
        d = pd.merge(left=d, right=c, on='name', how='outer')
        
    if idx == 'EVI':
        EVI = d.copy()
    else : NDVI = d.copy()

fin = pd.merge(EVI, NDVI, on='name', how='outer')
#fin.info()

In [6]:
def merge_mean_std(path, district_info,resolution):
    import os
    import pandas as pd
    import numpy as np
    """
    fuction that merges mean file and std file using district_code_key.
    
        * path = where the files in
        * file_list = mean+std / EVI+NDVI
        * district_code_key = df having two columns [district_name, district_code]
        * resolution = scale of region (sido, sigungu, dong, buffer200m, buffer500m, buffer1km)
        
    """   
    admin = [c for c in district_info.columns if resolution in c]
    admin_code = [c for c in admin if 'CD' in c][0]
    
    if resolution == 'sido':
        district_info = district_info.sort_values(by=admin_code, axis=0).rename(columns={f'{resolution}_CD':'code', 'sido_name':'name'})
    elif resolution == 'sigungu':        
        district_info = district_info.sort_values(by=admin_code, axis=0).rename(columns={f'{resolution}_CD':'code', 'sidosigungu_name':'name'})
    elif resolution == 'dong':        
        district_info = district_info.sort_values(by=admin_code, axis=0).rename(columns={f'{resolution}_CD':'code', 'sidosigungudong_name':'name'})
    else:
        raise Exception('올바른 지역 단위를 입력하세요.')
    
    
    file_list = os.listdir(path)
    file_list.sort()
    
    for index in ['EVI','NDVI']:
        
        print(f'\n\n*************{index} green index:************')
        
        RESULTs_list = district_info[['name','code']].drop_duplicates(['code']).copy().sort_values(by='name',ascending=True)
        RESULT = district_info[['name','code']].drop_duplicates(['code']).copy()
        
        RESULT['key'] = RESULT.apply(lambda x: str(x['name']) + str(x['code']), axis=1)
        
        mean_list = [c for c in file_list if 'mean' in c and index in c]
  
        for i, m in enumerate(mean_list):
            print(f'{m} merge START')
            
            # 특정 yearmonth column
            m_date = m.split('_')[0]
        
            std_list = [c for c in file_list if f'std_{index}' in c and c.startswith(m_date)]

            mean = pd.read_csv(path+'//'+m)
            std = pd.read_csv(path+'//'+std_list[0])
            
            #print(f'{index} {m_date} -> mean data shape: ', mean.shape)
            #print(f'{index} {m_date} -> std data shape: ', std.shape)
            
            print(f'--shape: {mean.shape}, mean data == std data? {mean.shape == std.shape}')
        
            mean_code = [code for code in mean.columns  if 'CD' in str(code)][0]
            std_code = [code for code in std.columns  if 'CD' in str(code)][0]
            
            if mean_code != std_code:
                raise Exception('mean과 std 컬럼명이 일치하지 않습니다.')
            
            mean_std = pd.merge(mean, std, on=mean_code, how='outer').rename(columns={mean_code:'code', 'mean':'average', 'stdDev':'stdev'})
            #print(f'df shape after merge mean + std: ', mean_std.shape)
            
            data = pd.merge(left=mean_std, right=RESULT, on='code', how='left')
            
            data['key'] = data.apply(lambda x: str(x['name']) + str(x['code']), axis=1)
            
            result = pd.DataFrame({f'{index}_{m_date}_{resolution}_avg':data.average, f'{index}_{m_date}_{resolution}_std':data.stdev})
            
            result[['name','code']] = data[['name','code']]
            result = result.sort_values(by='code', axis=0, ascending=True)
            
            RESULTs_list = pd.merge(left=RESULTs_list, right=result.drop(columns=['name']), on='code', how='left').sort_values(by='name', ascending=True)
            
            print(f'{index}: {m} COMPLETE \n')
        
        if index == 'EVI':
            EVI = RESULTs_list.copy()
        else:
            NDVI = RESULTs_list.copy()
        
    merged_data= pd.merge(EVI, NDVI.drop(columns=['name']), on='code', how='outer')
    merged_data= pd.merge(left=merged_data, right=district_info.drop(columns='name'), on='code', how='outer')
      
    return merged_data

In [28]:
# 2015년 
sigungu_2015 = merge_mean_std(path='.//2015년//2015_sigungu', resolution='sigungu', district_info=district)
dong_2015 = merge_mean_std(path='.//2015년//2015_dong', resolution='dong', district_info=district)



*************EVI green index:************
201501_mean_EVI_by_sigungu.csv merge START
--shape: (252, 2), mean data == std data? True
EVI: 201501_mean_EVI_by_sigungu.csv COMPLETE 

201502_mean_EVI_by_sigungu.csv merge START
--shape: (252, 2), mean data == std data? True
EVI: 201502_mean_EVI_by_sigungu.csv COMPLETE 

201503_mean_EVI_by_sigungu.csv merge START
--shape: (252, 2), mean data == std data? True
EVI: 201503_mean_EVI_by_sigungu.csv COMPLETE 

201504_mean_EVI_by_sigungu.csv merge START
--shape: (252, 2), mean data == std data? True
EVI: 201504_mean_EVI_by_sigungu.csv COMPLETE 

201505_mean_EVI_by_sigungu.csv merge START
--shape: (252, 2), mean data == std data? True
EVI: 201505_mean_EVI_by_sigungu.csv COMPLETE 

201506_mean_EVI_by_sigungu.csv merge START
--shape: (252, 2), mean data == std data? True
EVI: 201506_mean_EVI_by_sigungu.csv COMPLETE 

201507_mean_EVI_by_sigungu.csv merge START
--shape: (252, 2), mean data == std data? True
EVI: 201507_mean_EVI_by_sigungu.csv COMPLETE

In [29]:
fin = fin.rename(columns={'name':'sido_name'})

fin['sido_name'] = fin.sido_name.replace({'Chungcheongbuk-do':'충청북도','Chungcheongnam-do':'충청남도', 'Gangwon-do':'강원도','Gyeonggi-do':'경기도',
                             'Gyeongsangbuk-do':'경상북도', 'Gyeongsangnam-do':'경상남도', 'Jeju':'제주특별자치도', 'Jeollabuk-do':'전라북도', 'Jeollanam-do':'전라남도',
                             'Busan':'부산광역시', 'Daegu':'대구광역시', 'Daejeon':'대전광역시', 'Gwangju':'광주광역시', 'Incheon':'인천광역시', 'Ulsan':'울산광역시',
                             'Seoul':'서울특별시', 'Sejong':'세종특별자치시'})

In [30]:
sido_2015 = pd.merge(left=fin, right=district, on='sido_name',how='outer')
sido_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3502 entries, 0 to 3501
Data columns (total 56 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sido_name             3502 non-null   object 
 1   EVI_201501_SIDO_avg   2431 non-null   float64
 2   EVI_201501_SIDO_std   2431 non-null   float64
 3   EVI_201502_SIDO_avg   2383 non-null   float64
 4   EVI_201502_SIDO_std   2383 non-null   float64
 5   EVI_201503_SIDO_avg   3197 non-null   float64
 6   EVI_201503_SIDO_std   3197 non-null   float64
 7   EVI_201504_SIDO_avg   3502 non-null   float64
 8   EVI_201504_SIDO_std   3502 non-null   float64
 9   EVI_201505_SIDO_avg   3459 non-null   float64
 10  EVI_201505_SIDO_std   3459 non-null   float64
 11  EVI_201506_SIDO_avg   1840 non-null   float64
 12  EVI_201506_SIDO_std   1840 non-null   float64
 13  EVI_201507_SIDO_avg   332 non-null    float64
 14  EVI_201507_SIDO_std   332 non-null    float64
 15  EVI_201508_SIDO_avg  

### **Merge total regions in 2015**

In [31]:
sido_2015 = sido_2015.drop(columns=['sidosigungu_name','sidosigungudong_name'])
sigungu_2015 = sigungu_2015.rename(columns={'code':'sigungu_CD'}).drop(columns=['name','sidosigungudong_name'])
dong_2015 = dong_2015.rename(columns={'code':'dong_CD'}).drop(columns=['name','sidosigungu_name'])

In [32]:
sido_2015= sido_2015.loc[:,sorted(sido_2015.columns.to_list())].sort_values(by=['sido_CD', 'sigungu_CD', 'dong_CD'])
sigungu_2015 = sigungu_2015.loc[:,sorted(sigungu_2015.columns.to_list())].sort_values(by=['sido_CD', 'sigungu_CD', 'dong_CD'])
dong_2015 = dong_2015.loc[:,sorted(dong_2015.columns.to_list())].sort_values(by=['sido_CD', 'sigungu_CD', 'dong_CD'])

a = sido_2015['dong_CD'].values == sigungu_2015['dong_CD'].values 
b = sigungu_2015['dong_CD'].values == dong_2015['dong_CD'].values
c = sido_2015['dong_CD'].values == dong_2015['dong_CD'].values

d= pd.DataFrame({'a':a,'b':b,'c':c})
d.all(axis=0)

a    True
b    True
c    True
dtype: bool

**SIDO**

In [33]:
idx = ["sido_CD","sido_name","sigungu_CD","sigungu_name","dong_CD","dong_name"]
avg = [c for c in sido_2015.columns.to_list() if 'avg' in c]
std = [c for c in sido_2015.columns.to_list() if 'std' in c]

sido_2015 = sido_2015.drop_duplicates(['sido_CD'], keep='first')

sido_2015_avg = sido_2015.loc[:, idx+avg].drop(columns=['sigungu_CD','sigungu_name','dong_CD','dong_name'])
sido_2015_std = sido_2015.loc[:, idx+std].drop(columns=['sigungu_CD','sigungu_name','dong_CD','dong_name'])

**SIGUNGU**

In [35]:
idx = ["sido_CD","sido_name","sigungu_CD","sigungu_name","dong_CD","dong_name"]
avg = [c for c in sigungu_2015.columns.to_list() if 'avg' in c]
std = [c for c in sigungu_2015.columns.to_list() if 'std' in c]

sigungu_2015 = sigungu_2015.drop_duplicates(['sigungu_CD'], keep='first')

sigungu_2015_avg = sigungu_2015.loc[:, idx+avg].drop(columns=['dong_CD','dong_name'])
sigungu_2015_std = sigungu_2015.loc[:, idx+std].drop(columns=['dong_CD','dong_name'])

**DONG**

In [36]:
idx = ["sido_CD","sido_name","sigungu_CD","sigungu_name","dong_CD","dong_name"]
avg = [c for c in dong_2015.columns.to_list() if 'avg' in c]
std = [c for c in dong_2015.columns.to_list() if 'std' in c]

dong_2015_avg = dong_2015.loc[:, idx+avg]
dong_2015_std = dong_2015.loc[:, idx+std]

**SIDO+SIGUNGU+DONG = 하나의 엑셀 파일에 저장**

In [37]:
#엑셀 파일 열기
writer = pd.ExcelWriter('2015_merged_data.xlsx', engine='openpyxl')

#sheet별 작성
sido_2015_avg.to_excel(writer, header=True, index=False,encoding='cp949', sheet_name='시도_mean')
sido_2015_std.to_excel(writer, header=True, index=False,encoding='cp949', sheet_name='시도_std')
sigungu_2015_avg.to_excel(writer, header=True, index=False,encoding='cp949', sheet_name='시군구_mean')
sigungu_2015_std.to_excel(writer, header=True, index=False,encoding='cp949', sheet_name='시군구_std')
dong_2015_avg.to_excel(writer, header=True, index=False,encoding='cp949', sheet_name='읍면동_mean')
dong_2015_std.to_excel(writer, header=True, index=False,encoding='cp949', sheet_name='읍면동_std')

#작성 완료 후 저장
writer.save()

문제점

In [ ]:
# 문제점 파악: district_name만을 가져오면, 겹치는 변수가 생길 수 있음 ex. 부산 동구 == 대구 동구

a = pd.read_csv('.//2015년//2015_sigungu//201501_mean_EVI_by_sigungu.csv')
admin_subset = district[['sigungu_name','sigungu_CD']]
admin_subset['sigungu_key'] = admin_subset.apply(lambda x: str(x['sigungu_name']) + str(x['sigungu_CD']), axis=1)

print('name unique 갯수:',admin_subset.sigungu_name.nunique())
print('code unique 갯수:',admin_subset.sigungu_CD.nunique())
print('key unique 갯수:',admin_subset.sigungu_key.nunique())

b= admin_subset.drop_duplicates(['sigungu_name'])
c= admin_subset.drop_duplicates(['sigungu_CD'])

print('name 기준: ', admin_subset.drop_duplicates(['sigungu_name']).shape)
print('code 기준: ', admin_subset.drop_duplicates(['sigungu_CD']).shape)

diff = list(set(c.sigungu_key) - set(b.sigungu_key))

diff_subset = admin_subset.loc[[True if c in diff else False for c in admin_subset['sigungu_key']], :]
diff_subset.drop_duplicates(['sigungu_key']).sort_values(by='sigungu_name')